In [6]:
# Import packages.
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import json

## Get NASA Mars News.

In [7]:
# Scrape data from nasa.gov API.
news_url = "https://mars.nasa.gov/api/v1/news_items/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
resp = requests.get(news_url, headers=header)
resp = resp.json()


In [8]:
# Check out the response.
print(resp.keys())
print(resp)

[u'per_page', u'items', u'total', u'page', u'more']
{u'per_page': 40, u'items': [{u'body': u'<p>Several members of the Mars InSight team accepted a proclamation on behalf of the mission from L.A. County Board of Supervisors on Tuesday, Feb. 19.</p>\n\n<p>"I\'ve always wanted to say this: \'mission accomplished.\'" That\'s how 5th District Supervisor Kathryn Barger began the presentation. She said, "I want to thank the incredible team at JPL, who has gone above and beyond to really propel Los Angeles County into the future as a leader in science and technology."</p>\n\n<p>InSight Project Manager Tom Hoffman noted that although all five team members present at the ceremony live within Barger\'s district, the team includes people from all over L.A. County, other states and other countries. Hoffman said he was proud to represent all of them.</p>\n\n<p>"It really takes more than a village to build a spacecraft; it takes an entire planet to build this particular spacecraft," Hoffman said.</p

In [9]:
# Extract relevant data.
articles = resp.get('items')
articles

[{u'body': u'<p>Several members of the Mars InSight team accepted a proclamation on behalf of the mission from L.A. County Board of Supervisors on Tuesday, Feb. 19.</p>\n\n<p>"I\'ve always wanted to say this: \'mission accomplished.\'" That\'s how 5th District Supervisor Kathryn Barger began the presentation. She said, "I want to thank the incredible team at JPL, who has gone above and beyond to really propel Los Angeles County into the future as a leader in science and technology."</p>\n\n<p>InSight Project Manager Tom Hoffman noted that although all five team members present at the ceremony live within Barger\'s district, the team includes people from all over L.A. County, other states and other countries. Hoffman said he was proud to represent all of them.</p>\n\n<p>"It really takes more than a village to build a spacecraft; it takes an entire planet to build this particular spacecraft," Hoffman said.</p>\n\n<p><b>News Media Contact</b></p>\n\n<p><span>Andrew Good</span><br />\n<spa

In [10]:
article_df = pd.DataFrame.from_records(articles)
article_df.head()

,body,byline,date,description,id,item_name,main_image,main_image_alt,meta_author,meta_keywords,missions,news_type,status,target,thumb,title,updated_at,url
0,<p>Several members of the Mars InSight team ac...,,"February 25, 2019",Several members of the Mars InSight team accep...,8417,,/system/news_items/main_images/8417_insight201...,L.A. County Supervisor Kathryn Barger (center)...,,,"[[183, insight, InSight]]","[[165, feature, Feature Stories]]",1,_self,/system/news_items/list_view_images/8417_insig...,InSight Lands Praise and a Proclamation from L...,2019-02-26T17:43:52.723Z,/news/8417/insight-lands-praise-and-a-proclama...
1,"<p>NASA's Curiosity rover is busy <a href=""htt...",,"February 22, 2019",NASA's Mars rover Curiosity is in good health ...,8416,,/system/news_items/main_images/8416_PIA23047.jpg,NASA's Curiosity Mars took this image with its...,,,"[[175, mep, Mars Exploration Program], [176, m...","[[204, status_reports, Status Reports]]",1,_self,/system/news_items/list_view_images/8416_PIA23...,"After a Reset, Curiosity Is Operating Normally",2019-02-22T21:36:30.446Z,/news/8416/after-a-reset-curiosity-is-operatin...
2,"<p>No matter how cold your winter has been, it...",,"February 19, 2019","By collecting data around the clock, NASA's la...",8415,,/system/news_items/main_images/8415_PIA22876-m...,The white east- and west-facing booms — called...,,,"[[183, insight, InSight]]","[[165, feature, Feature Stories]]",1,_self,/system/news_items/list_view_images/8415_PIA22...,InSight Is the Newest Mars Weather Service,2019-02-19T17:02:01.949Z,/news/8415/insight-is-the-newest-mars-weather-...
3,"<p>After 15 years, the mission of NASA's Oppor...",,"February 13, 2019",Opportunity's mission is complete. Here are hi...,8414,,/system/news_items/main_images/8414_1_MAIN_mar...,This scene from the panoramic camera on NASA's...,,,"[[175, mep, Mars Exploration Program], [177, m...","[[165, feature, Feature Stories]]",1,_self,/system/news_items/list_view_images/8414_1_MAI...,Six Things to Know About NASA's Opportunity Rover,2019-02-13T19:22:15.859Z,/news/8414/six-things-to-know-about-nasas-oppo...
4,<section class='carousel_module ms_carousel wi...,,"February 13, 2019",NASA's Opportunity Mars rover mission is compl...,8413,,/system/news_items/main_images/8413_PIA06739-1...,Opportunity Catches its Shadow,,,"[[175, mep, Mars Exploration Program], [177, m...","[[19, press, Press Releases]]",1,_self,/system/news_items/list_view_images/8413_PIA06...,NASA's Opportunity Rover Mission on Mars Comes...,2019-02-15T17:30:20.664Z,/news/8413/nasas-opportunity-rover-mission-on-...


In [11]:
# Store title and content for later.
titles = article_df['title']
content = article_df['body']
titles_content = article_df[['title','body']]

## Get JPL Mars Space Images - Featured Image.

In [12]:
# Image url.
image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
    
# Use splinter to navigate the site and find the image url for the current Featured 
# Mars Image and assign the url string to a variable called featured_image_url.
    
executable_path = {'executable_path': '/Users/jenniferwilson/Desktop/Repositories/UDEN201811DATA3-Homework/chromedriver'}
browser = Browser('chrome', **executable_path)

browser.visit(image_url)
html = browser.html
soup = bs(html, 'html.parser')

# How are these different
#soup.find_all("article",{"class":"carousel_item"})[0]
#soup.find_all("article",{"class":"carousel_item"})[0].contents

In [15]:
# From soup object above, get featured image URL.
featured_image_url = "https://www.jpl.nasa.gov" + soup.article["style"].split("'")[1]
featured_image_url

u'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA19046-1920x1200.jpg'

## Get Mars Weather.

In [29]:
twitter_url = "https://twitter.com/marswxreport?lang=en"
header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
resp = requests.get(twitter_url, headers=header)
soup = bs(resp.text,'html.parser')

In [30]:
# Extract first tweet.
tweet_text = soup.find("p",{"class":"TweetTextSize"}).get_text()

# Remove ASCII characters.
tweet_text = tweet_text.encode("ascii","ignore")

# Split based on line breaks.
mars_weather = tweet_text.splitlines()

mars_weather

# Output should look like this:
#InSight sol 88 (2019-02-25) low -95.2ºC (-139.4ºF) high -17.8ºC (0.0ºF)
#winds from the SW at 5.2 m/s (11.6 mph) gusting to 19.8 m/s (44.3 mph)
#pressure at 7.20 hPa


['InSight sol 88 (2019-02-25) low -95.2C (-139.4F) high -17.8C (0.0F)',
 'winds from the SW at 5.2 m/s (11.6 mph) gusting to 19.8 m/s (44.3 mph)',
 'pressure at 7.20 hPapic.twitter.com/1msjBvhiu7']

## Mars Facts

In [53]:
# use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

facts_url = "http://space-facts.com/mars/"
facts = pd.read_html(facts_url)
facts_df = facts[0]
facts_df.rename(columns={0: "Stat", 1:"Values"}, inplace = True) 
#facts_df['Values'] = map(lambda x: x.encode('utf-8'), facts_df["Values"])
facts_df['Values'] = map(lambda x: x.encode('ascii', 'ignore').decode('ascii'), facts_df["Values"])
facts_df

,Stat,Values
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [58]:
# Use Pandas to convert the data to a HTML table string.
# https://stackoverflow.com/questions/3206344/passing-html-to-template-using-flask-jinja2
facts_html = facts_df.to_html()
print(facts_html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Stat</th>
      <th>Values</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.42 x 10^23 kg (10.7% Earth)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.52 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-153 to 20 C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</th>
      <td>

In [57]:
%%bash
open . 